# 📗 발표용 Part 3: 모델링 및 최적화 v2 개선판

## 🎯 v2 개선판 주요 변경 사항

### v2 대비 개선 사항 (P0/P1 모두 반영)

**P0 (필수 수정):**
1. ✅ **Pipeline 구조 도입** - ImbPipeline 사용, 전처리+리샘플링+모델 통합
2. ✅ **RandomizedSearchCV n_iter=200** - 탐색 성능 향상
3. ✅ **앙상블 다양성 체크** - Top 3가 모두 GBM이면 이종 모델 강제 포함

**P1 (권장 개선):**
4. ✅ **CV 기반 임계값 최적화** - Validation + CV 평균 사용
5. ✅ **Winsorizer 실험** - 있음/없음 성능 비교
6. ✅ **Traffic Light Yellow 로직 일관성** - Recall 기준 일관 사용

---

## 🎯 Part 3 목표 및 이전 Part 요약

### 이전 Part 완료 사항

**Part 1: 문제 정의 및 핵심 발견**
- 50,105개 한국 기업, 부도율 1.51% (1:66 불균형)
- **핵심 발견**: 유동성이 가장 강력한 예측 변수
- 업종별 부도율 2배 차이 (제조업 vs 서비스업)

**Part 2: 도메인 특성 공학**
- 52개 도메인 기반 특성 생성
- VIF/IV/AUC 기반 특성 선택 → **27개 최종 특성**
- 7개 카테고리: 유동성, 지급불능, 재무조작, 한국시장, 이해관계자, 복합리스크, 상호작용

### Part 3 v2 목표 (Data Leakage 완전 제거)

**🚨 핵심 개선: Data Leakage 완전 제거**

```
✅ v2 해결책:
- 3-Way Split (Train/Validation/Test)
- Test set은 최종 평가 직전 단 한 번만
- 모든 의사결정은 Validation set 기반
- Pipeline 구조로 전처리/리샘플링 누락 방지
```

**주요 개선 사항:**
1. **3-Way Data Split** (Train 60% / Validation 20% / Test 20%)
2. **Pipeline 기반 리샘플링 전략** (SMOTE vs Class Weight)
3. **Validation 기반 모델 선택** + Statistical Test + Ensemble Diversity
4. **CV+Validation 임계값 최적화** (F2-Score, Recall 우선)
5. **데이터 기반 Traffic Light** (Recall 80%/95% 보장)
6. **Test Set은 마지막 평가만** (절대 의사결정에 사용 안 함)

**목표 성능:**
- PR-AUC: 0.15~0.20 (불균형 데이터 고려)
- F2-Score: 0.35~0.50 (Recall 우선)
- Recall: 60~80%
- Type II Error: 20~40% (부도 미탐지 최소화)

---

## 0. 환경 설정

In [ ]:
# 기본 라이브러리
import pandas as pd
import numpy as np
import warnings
import platform
import os
import joblib
import pickle
from datetime import datetime

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# 머신러닝 - 전처리
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin

# 머신러닝 - 모델
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier

In [ ]:
# 머신러닝 - 튜닝 (⭐ Pipeline 사용)
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline  # ⭐ Imbalanced Pipeline

# 불균형 데이터 처리
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN
from imblearn.combine import SMOTETomek, SMOTEENN

In [ ]:
# 평가 메트릭
from sklearn.metrics import (
    classification_report, confusion_matrix,
    roc_auc_score, average_precision_score,
    precision_recall_curve, roc_curve,
    make_scorer, fbeta_score, recall_score, precision_score
)

# 통계 검정
from scipy.stats import wilcoxon, ttest_rel

In [ ]:
# 경고 무시
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (CLAUDE.md 준수)
if platform.system() == 'Darwin':
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')
else:
    plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)

# 시각화 설정
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# 랜덤 시드 (재현성)
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [ ]:
# sklearn, catboost 버전 임포트
import sklearn
import catboost

print("✅ 환경 설정 완료")
print(f"   - NumPy: {np.__version__}")
print(f"   - Pandas: {pd.__version__}")
print(f"   - Scikit-learn: {sklearn.__version__}")
print(f"   - LightGBM: {lgb.__version__}")
print(f"   - XGBoost: {xgb.__version__}")
print(f"   - CatBoost: {catboost.__version__}")
print(f"   - Platform: {platform.system()}")
print(f"   - Random State: {RANDOM_STATE}")

## 1. 데이터 로딩 및 3-Way Split ⭐

### 1.1 데이터 로딩

In [ ]:
# 데이터 경로 (하드코딩 금지)
DATA_DIR = '../data'
FEATURES_FILE = os.path.join(DATA_DIR, 'features', 'domain_based_features_완전판.csv')
PROCESSED_DIR = os.path.join(DATA_DIR, 'processed')

# 디렉토리 생성
os.makedirs(PROCESSED_DIR, exist_ok=True)

# 데이터 로딩
print(f"📂 데이터 로딩 중: {FEATURES_FILE}")
df = pd.read_csv(FEATURES_FILE, encoding='utf-8')

print(f"✅ 데이터 로딩 완료")
print(f"   - 전체 기업 수: {len(df):,}")
print(f"   - 전체 변수 수: {len(df.columns)}")
print(f"   - 메모리 사용: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# 데이터 미리보기
df.head(3)

In [ ]:
# 타겟 변수 확인
TARGET_COL = '모형개발용Performance(향후1년내부도여부)'

print(f"🎯 타겟 변수: {TARGET_COL}")
print(f"\n부도 분포:")
print(df[TARGET_COL].value_counts())
print(f"\n부도율: {df[TARGET_COL].mean():.4%}")
print(f"불균형 비율: 1:{int(1/df[TARGET_COL].mean())}")

### 1.2 특성 및 타겟 분리

In [ ]:
# 특성(X)과 타겟(y) 분리
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

print(f"✅ 특성 및 타겟 분리 완료")
print(f"   - X shape: {X.shape}")
print(f"   - y shape: {y.shape}")
print(f"   - 특성 목록 (27개):")
for i, col in enumerate(X.columns, 1):
    print(f"      {i:2d}. {col}")

### 1.3 3-Way Split: Train / Validation / Test ⭐⭐⭐

**🚨 Data Leakage 방지를 위한 핵심 설계:**

```
전체 데이터 (50,105)
├─ Train Set (60%, ~30,063): 모델 학습 + CV 튜닝
├─ Validation Set (20%, ~10,021): 모델 선택, 임계값 최적화, 의사결정
└─ Test Set (20%, ~10,021): 최종 평가만 (절대 건드리지 않음!)
```

In [ ]:
# 1차 분할: Train+Val (80%) vs Test (20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    stratify=y, 
    random_state=RANDOM_STATE
)

# 2차 분할: Train (60%) vs Validation (20%)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, 
    test_size=0.25, 
    stratify=y_temp, 
    random_state=RANDOM_STATE
)

print("="*70)
print("✅ 3-Way Split 완료")
print("="*70)
print(f"Train Set:      {len(X_train):,} ({len(X_train)/len(X)*100:.1f}%, 부도율: {y_train.mean():.4%})")
print(f"Validation Set: {len(X_val):,} ({len(X_val)/len(X)*100:.1f}%, 부도율: {y_val.mean():.4%})")
print(f"Test Set:       {len(X_test):,} ({len(X_test)/len(X)*100:.1f}%, 부도율: {y_test.mean():.4%})")
print("="*70)
print("⚠️  Test Set은 최종 평가 전까지 절대 사용하지 않음!")
print("="*70)

In [ ]:
# 불균형 비율 계산 (Class Weight용)
n_pos = y_train.sum()
n_neg = len(y_train) - n_pos
imbalance_ratio = n_neg / n_pos

print(f"📊 Train Set 불균형 분석")
print(f"="*70)
print(f"부도 기업 (Positive):  {n_pos:,} ({n_pos/len(y_train)*100:.2f}%)")
print(f"정상 기업 (Negative):  {n_neg:,} ({n_neg/len(y_train)*100:.2f}%)")
print(f"불균형 비율:           1:{imbalance_ratio:.1f}")
print(f"="*70)
print(f"\nClass Weight 계산:")
print(f"  - sqrt(neg/pos) = {np.sqrt(imbalance_ratio):.2f}")
print(f"  - neg/pos = {imbalance_ratio:.2f}")

## 2. 전처리 파이프라인 정의

### 2.1 Custom Transformers

In [ ]:
# Custom Transformer 1: 무한대 값 처리
class InfiniteHandler(BaseEstimator, TransformerMixin):
    """무한대 값을 0으로 대체"""
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        X = X.replace([np.inf, -np.inf], 0)
        return X

print("✅ InfiniteHandler 정의 완료")

In [ ]:
# Custom Transformer 2: Winsorizer (이상치 제한)
class Winsorizer(BaseEstimator, TransformerMixin):
    """이상치를 percentile 범위로 제한"""
    
    def __init__(self, lower=0.005, upper=0.995):
        self.lower = lower
        self.upper = upper
        self.lower_bounds_ = None
        self.upper_bounds_ = None
    
    def fit(self, X, y=None):
        self.lower_bounds_ = np.percentile(X, self.lower * 100, axis=0)
        self.upper_bounds_ = np.percentile(X, self.upper * 100, axis=0)
        return self
    
    def transform(self, X):
        X = X.copy()
        X = np.clip(X, self.lower_bounds_, self.upper_bounds_)
        return X

print("✅ Winsorizer 정의 완료")

### 2.2 파이프라인 생성 함수

In [ ]:
def create_preprocessing_pipeline(use_winsorizer=False):
    """
    전처리 파이프라인 생성
    
    순서: InfiniteHandler → Imputer → Winsorizer(선택) → Scaler
    
    Parameters:
    -----------
    use_winsorizer : bool, default=False
        Winsorizer 적용 여부
    """
    steps = [
        ('inf_handler', InfiniteHandler()),
        ('imputer', SimpleImputer(strategy='median')),
    ]
    
    if use_winsorizer:
        steps.append(('winsorizer', Winsorizer(0.005, 0.995)))
    
    steps.append(('scaler', RobustScaler()))
    
    return Pipeline(steps)

print("✅ 전처리 파이프라인 생성 함수 정의 완료")

### 2.3 P1 개선: Winsorizer 실험 ⭐

**목적:** Tree 모델은 이상치에 강건하지만, 부도 데이터의 극단값이 중요 시그널일 수 있음

**실험:** Winsorizer 있음 vs 없음 성능 비교 (Validation Set)

In [ ]:
# Winsorizer 실험 (LightGBM 빠른 테스트)
print("📊 Winsorizer 실험 (Validation Set 기반)")
print("="*70)

winsorizer_results = {}

for use_wins in [False, True]:
    label = "Winsorizer 있음" if use_wins else "Winsorizer 없음"
    
    # 전처리
    prep = create_preprocessing_pipeline(use_winsorizer=use_wins)
    X_train_prep = prep.fit_transform(X_train)
    X_val_prep = prep.transform(X_val)
    
    # LightGBM 학습
    model = lgb.LGBMClassifier(
        n_estimators=100,
        learning_rate=0.05,
        max_depth=5,
        scale_pos_weight=np.sqrt(imbalance_ratio),
        random_state=RANDOM_STATE,
        n_jobs=-1,
        verbose=-1
    )
    model.fit(X_train_prep, y_train)
    
    # Validation 평가
    y_val_prob = model.predict_proba(X_val_prep)[:, 1]
    pr_auc = average_precision_score(y_val, y_val_prob)
    
    winsorizer_results[label] = pr_auc
    print(f"{label:20s} | Validation PR-AUC: {pr_auc:.4f}")

print("="*70)

# 선택
if winsorizer_results["Winsorizer 있음"] > winsorizer_results["Winsorizer 없음"]:
    use_winsorizer_final = True
    diff = winsorizer_results["Winsorizer 있음"] - winsorizer_results["Winsorizer 없음"]
    print(f"✅ 선택: Winsorizer 사용 (PR-AUC +{diff:.4f})")
else:
    use_winsorizer_final = False
    diff = winsorizer_results["Winsorizer 없음"] - winsorizer_results["Winsorizer 있음"]
    print(f"✅ 선택: Winsorizer 미사용 (PR-AUC +{diff:.4f})")
    print(f"   이유: Tree 모델은 이상치에 강건, 극단값이 부도 시그널일 수 있음")

print("="*70)

## 3. P0 개선: Pipeline 기반 리샘플링 전략 ⭐⭐⭐

### 3.1 개선 사항

**v2 문제점:**
- 전처리와 모델이 분리 → 재사용 시 오류 위험
- 리샘플링을 Pipeline에 통합하지 않음

**v2 개선판 해결책:**
- ✅ **ImbPipeline** 사용: 전처리 + 리샘플링 + 모델 통합
- ✅ Raw 데이터로 AutoML (전처리 누락 방지)
- ✅ Best practice 준수

### 3.2 Pipeline 생성 함수

In [ ]:
def create_model_pipeline(classifier, resampler='passthrough', use_winsorizer=False):
    """
    전체 모델 파이프라인 생성 (전처리 + 리샘플링 + 모델)
    
    Parameters:
    -----------
    classifier : estimator
        분류 모델
    resampler : 'passthrough' or sampler object
        리샘플링 방법 (passthrough=없음)
    use_winsorizer : bool
        Winsorizer 사용 여부
    
    Returns:
    --------
    pipeline : ImbPipeline
    """
    # 전처리 단계
    steps = [
        ('inf_handler', InfiniteHandler()),
        ('imputer', SimpleImputer(strategy='median')),
    ]
    
    if use_winsorizer:
        steps.append(('winsorizer', Winsorizer(0.005, 0.995)))
    
    steps.append(('scaler', RobustScaler()))
    
    # 리샘플링 + 모델
    steps.append(('resampler', resampler))
    steps.append(('classifier', classifier))
    
    return ImbPipeline(steps)

print("✅ Pipeline 생성 함수 정의 완료")

### 3.3 리샘플링 전략 대조 실험 (Pipeline 기반)

In [ ]:
print("📊 리샘플링 전략 대조 실험 (Pipeline 기반)")
print("="*70)
print("Strategy A: SMOTE 계열 (Class Weight 없음)")
print("Strategy B: Class Weight Only (리샘플링 없음)")
print("="*70)

In [ ]:
# Strategy A: SMOTE 계열
print("\n🔵 Strategy A: SMOTE 계열")
print("-"*70)

smote_methods = {
    'SMOTE': SMOTE(sampling_strategy=0.2, random_state=RANDOM_STATE),
    'BorderlineSMOTE': BorderlineSMOTE(sampling_strategy=0.2, random_state=RANDOM_STATE),
    'SMOTETomek': SMOTETomek(sampling_strategy=0.2, random_state=RANDOM_STATE)
}

strategy_a_results = {}

for name, sampler in smote_methods.items():
    # Pipeline 생성 (Class Weight 없음)
    pipeline = create_model_pipeline(
        classifier=lgb.LGBMClassifier(
            n_estimators=100,
            learning_rate=0.05,
            max_depth=5,
            scale_pos_weight=1,  # Class Weight 없음
            random_state=RANDOM_STATE,
            n_jobs=-1,
            verbose=-1
        ),
        resampler=sampler,
        use_winsorizer=use_winsorizer_final
    )
    
    # ⭐ Raw 데이터로 학습 (전처리는 Pipeline 내부에서)
    pipeline.fit(X_train, y_train)
    
    # Validation 평가
    y_val_prob = pipeline.predict_proba(X_val)[:, 1]
    pr_auc = average_precision_score(y_val, y_val_prob)
    
    strategy_a_results[name] = pr_auc
    print(f"{name:20s} | Validation PR-AUC: {pr_auc:.4f}")

print("-"*70)

In [ ]:
# Strategy B: Class Weight Only
print("\n🟢 Strategy B: Class Weight Only")
print("-"*70)

class_weight_methods = {
    'No Weight': 1,
    'Sqrt Weight': np.sqrt(imbalance_ratio),
    'Full Weight': imbalance_ratio
}

strategy_b_results = {}

for name, weight in class_weight_methods.items():
    # Pipeline 생성 (리샘플링 없음)
    pipeline = create_model_pipeline(
        classifier=lgb.LGBMClassifier(
            n_estimators=100,
            learning_rate=0.05,
            max_depth=5,
            scale_pos_weight=weight,  # Class Weight 사용
            random_state=RANDOM_STATE,
            n_jobs=-1,
            verbose=-1
        ),
        resampler='passthrough',  # 리샘플링 없음
        use_winsorizer=use_winsorizer_final
    )
    
    # Raw 데이터로 학습
    pipeline.fit(X_train, y_train)
    
    # Validation 평가
    y_val_prob = pipeline.predict_proba(X_val)[:, 1]
    pr_auc = average_precision_score(y_val, y_val_prob)
    
    strategy_b_results[name] = pr_auc
    print(f"{name:20s} | Validation PR-AUC: {pr_auc:.4f} (weight={weight:.2f})")

print("-"*70)

In [ ]:
# 전략 선택
best_a_name = max(strategy_a_results, key=strategy_a_results.get)
best_a_pr_auc = strategy_a_results[best_a_name]

best_b_name = max(strategy_b_results, key=strategy_b_results.get)
best_b_pr_auc = strategy_b_results[best_b_name]

print("\n" + "="*70)
print("📊 리샘플링 전략 비교 결과 (Validation Set 기반)")
print("="*70)
print(f"Strategy A (SMOTE 계열) 최고:  {best_a_name:20s} | PR-AUC = {best_a_pr_auc:.4f}")
print(f"Strategy B (Class Weight) 최고: {best_b_name:20s} | PR-AUC = {best_b_pr_auc:.4f}")
print("="*70)

if best_b_pr_auc > best_a_pr_auc:
    diff = best_b_pr_auc - best_a_pr_auc
    pct = diff / best_a_pr_auc * 100
    selected_strategy = 'Class Weight'
    selected_weight = class_weight_methods[best_b_name]
    selected_resampler = 'passthrough'
    print(f"✅ 선택: Strategy B (Class Weight)")
    print(f"   - 이유: {diff:.4f} ({pct:.2f}%) 더 우수")
    print(f"   - scale_pos_weight: {selected_weight:.2f}")
else:
    diff = best_a_pr_auc - best_b_pr_auc
    pct = diff / best_b_pr_auc * 100
    selected_strategy = 'SMOTE'
    selected_weight = 1
    selected_resampler = smote_methods[best_a_name]
    print(f"✅ 선택: Strategy A (SMOTE 계열)")
    print(f"   - 이유: {diff:.4f} ({pct:.2f}%) 더 우수")
    print(f"   - 방법: {best_a_name}")

print("="*70)

## 4. P0 개선: AutoML 하이퍼파라미터 튜닝 (n_iter=200) ⭐

### 4.1 개선 사항

**v2 문제점:**
- n_iter=50~100 → 탐색 공간 대비 1% 미만

**v2 개선판:**
- ✅ **n_iter=200** (모델별)
- ✅ Pipeline 기반 AutoML
- ✅ Raw 데이터 사용

### 4.2 Scorer 정의

In [ ]:
# PR-AUC Scorer (주요 지표)
pr_auc_scorer = make_scorer(average_precision_score, needs_proba=True)

print("✅ Scorer 정의 완료")
print("   - PR-AUC: 불균형 데이터 핵심 지표")

### 4.3 파라미터 그리드 정의

In [ ]:
# 선택된 전략에 따른 파라미터 설정
if selected_strategy == 'Class Weight':
    # Strategy B: Class Weight만 사용
    resampler_options = ['passthrough']
    scale_pos_weight_options = [selected_weight]
    print(f"✅ 선택된 전략: Class Weight (scale_pos_weight={selected_weight:.2f})")
else:
    # Strategy A: SMOTE 사용
    resampler_options = [selected_resampler]
    scale_pos_weight_options = [1]
    print(f"✅ 선택된 전략: SMOTE ({best_a_name})")

In [ ]:
# LightGBM 파라미터 그리드
lgbm_param_grid = {
    'classifier__n_estimators': [50, 100, 200, 300],
    'classifier__learning_rate': [0.01, 0.03, 0.05, 0.1],
    'classifier__max_depth': [3, 5, 7, 10, -1],
    'classifier__num_leaves': [15, 31, 63, 127],
    'classifier__min_child_samples': [10, 20, 30, 50],
    'classifier__subsample': [0.6, 0.7, 0.8, 1.0],
    'classifier__colsample_bytree': [0.6, 0.7, 0.8, 1.0],
    'classifier__reg_alpha': [0, 0.01, 0.1, 1],
    'classifier__reg_lambda': [0, 0.01, 0.1, 1],
}

# XGBoost 파라미터 그리드
xgb_param_grid = {
    'classifier__n_estimators': [50, 100, 200, 300],
    'classifier__learning_rate': [0.01, 0.03, 0.05, 0.1],
    'classifier__max_depth': [3, 5, 7, 10],
    'classifier__min_child_weight': [1, 3, 5, 7],
    'classifier__subsample': [0.6, 0.7, 0.8, 1.0],
    'classifier__colsample_bytree': [0.6, 0.7, 0.8, 1.0],
    'classifier__gamma': [0, 0.1, 0.2, 0.5],
    'classifier__reg_alpha': [0, 0.01, 0.1, 1],
    'classifier__reg_lambda': [0, 0.01, 0.1, 1],
}

# CatBoost 파라미터 그리드
catboost_param_grid = {
    'classifier__iterations': [50, 100, 200, 300],
    'classifier__learning_rate': [0.01, 0.03, 0.05, 0.1],
    'classifier__depth': [3, 5, 7, 10],
    'classifier__l2_leaf_reg': [1, 3, 5, 7, 9],
}

# Logistic Regression 파라미터 그리드
lr_param_grid = {
    'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'classifier__penalty': ['l1', 'l2'],
    'classifier__solver': ['liblinear', 'saga'],
    'classifier__max_iter': [1000],
}

# Random Forest 파라미터 그리드
rf_param_grid = {
    'classifier__n_estimators': [50, 100, 200, 300],
    'classifier__max_depth': [5, 10, 15, 20, None],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__max_features': ['sqrt', 'log2', None],
}

print("✅ 파라미터 그리드 정의 완료")

### 4.4 모델 정의

In [ ]:
# 모델 정의 (Pipeline으로 wrapping)
models_to_tune = {
    'LightGBM': {
        'pipeline': create_model_pipeline(
            lgb.LGBMClassifier(
                scale_pos_weight=selected_weight,
                random_state=RANDOM_STATE,
                n_jobs=-1,
                verbose=-1
            ),
            resampler=selected_resampler if selected_strategy == 'SMOTE' else 'passthrough',
            use_winsorizer=use_winsorizer_final
        ),
        'param_grid': lgbm_param_grid,
        'n_iter': 200
    },
    'XGBoost': {
        'pipeline': create_model_pipeline(
            xgb.XGBClassifier(
                scale_pos_weight=selected_weight,
                random_state=RANDOM_STATE,
                n_jobs=-1,
                verbosity=0,
                tree_method='hist'
            ),
            resampler=selected_resampler if selected_strategy == 'SMOTE' else 'passthrough',
            use_winsorizer=use_winsorizer_final
        ),
        'param_grid': xgb_param_grid,
        'n_iter': 200
    },
    'CatBoost': {
        'pipeline': create_model_pipeline(
            CatBoostClassifier(
                scale_pos_weight=selected_weight,
                random_state=RANDOM_STATE,
                verbose=0
            ),
            resampler=selected_resampler if selected_strategy == 'SMOTE' else 'passthrough',
            use_winsorizer=use_winsorizer_final
        ),
        'param_grid': catboost_param_grid,
        'n_iter': 150  # CatBoost는 느리므로 150
    },
    'LogisticRegression': {
        'pipeline': create_model_pipeline(
            LogisticRegression(
                class_weight='balanced' if selected_strategy == 'Class Weight' else None,
                random_state=RANDOM_STATE,
                n_jobs=-1
            ),
            resampler=selected_resampler if selected_strategy == 'SMOTE' else 'passthrough',
            use_winsorizer=use_winsorizer_final
        ),
        'param_grid': lr_param_grid,
        'n_iter': 50
    },
    'RandomForest': {
        'pipeline': create_model_pipeline(
            RandomForestClassifier(
                class_weight='balanced' if selected_strategy == 'Class Weight' else None,
                random_state=RANDOM_STATE,
                n_jobs=-1
            ),
            resampler=selected_resampler if selected_strategy == 'SMOTE' else 'passthrough',
            use_winsorizer=use_winsorizer_final
        ),
        'param_grid': rf_param_grid,
        'n_iter': 100
    }
}

print("✅ 모델 Pipeline 생성 완료")
print(f"   - 전체 모델 수: {len(models_to_tune)}")

### 4.5 RandomizedSearchCV 실행 (⭐ n_iter=200)

In [ ]:
print("🚀 AutoML 하이퍼파라미터 튜닝 시작 (n_iter=200)")
print("="*70)
print("⚠️  Train Set + 5-Fold CV만 사용 (Validation/Test 미사용)")
print("⚠️  Raw 데이터 사용 (전처리는 Pipeline 내부에서)")
print("="*70)

search_results = {}
best_models = {}

for model_name, config in models_to_tune.items():
    print(f"\n{'='*70}")
    print(f"📊 {model_name} 튜닝 중... (n_iter={config['n_iter']})")
    print(f"{'='*70}")
    
    # RandomizedSearchCV
    search = RandomizedSearchCV(
        estimator=config['pipeline'],
        param_distributions=config['param_grid'],
        n_iter=config['n_iter'],
        scoring=pr_auc_scorer,
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE),
        n_jobs=-1,
        random_state=RANDOM_STATE,
        verbose=2,  # ⭐ 진행 상황 표시
        return_train_score=True
    )
    
    # ⭐ Raw 데이터로 학습
    search.fit(X_train, y_train)
    
    # 결과 저장
    search_results[model_name] = search
    best_models[model_name] = search.best_estimator_
    
    print(f"\n✅ {model_name} 완료")
    print(f"   - Best CV PR-AUC: {search.best_score_:.4f}")
    print(f"   - Best Params: {search.best_params_}")

print(f"\n{'='*70}")
print("✅ 모든 모델 튜닝 완료!")
print(f"{'='*70}")

### 4.6 결과 분석

In [ ]:
# 모델별 최고 성능
model_best_scores = {name: search.best_score_ for name, search in search_results.items()}

# 정렬
sorted_models = sorted(model_best_scores.items(), key=lambda x: x[1], reverse=True)

print("🏆 모델별 최고 CV 성능 (Train Set 5-Fold)")
print("="*70)
for rank, (name, score) in enumerate(sorted_models, 1):
    print(f"{rank}. {name:20s} | PR-AUC: {score:.4f}")
print("="*70)

In [ ]:
# 과적합 체크
print("\n📊 과적합 체크 (Train Score - CV Score)")
print("="*70)

for model_name, search in search_results.items():
    train_score = search.cv_results_['mean_train_score'][search.best_index_]
    cv_score = search.best_score_
    gap = train_score - cv_score
    
    status = "⚠️ 과적합 우려" if gap > 0.1 else "✅ 정상"
    
    print(f"{model_name:20s} | Train: {train_score:.4f} | CV: {cv_score:.4f} | "
          f"Gap: {gap:.4f} | {status}")

print("="*70)